<img src="./img/CCIT_Logo.png" alt="CCit Logo" width=400>

# 02 - Intro to SCA (Side-Channel Analysis) 🕵️

## ⚠️ Prerequisites

Hol' up - before continuing, ensure you have done the following:

* ☑ Read the introduction to Jupyter Notebooks, Matplotlib and the CyberChallenge Library in the [01-Learning to fly](./01-Learning%20to%20fly.ipynb) notebook.

### 📑 Summary

*In this lab, you'll learn what is Side-Channel Analysis, AKA **SCA**. You will understand why it is such an important attack to be aware of, both as attackers and defenders. You'll learn what kind of exploits such attack is leveraging, what are the targets and tons of examples. I'll give you a good introduction to Power Analysis and what are the basics behind these attacks. Finally, we will learn about the tool we are going to use in this course starting with the next lecture.*

### 💬 Learning outcomes
* Why SCA is so important
* What are the main sources of information leakage
* What is Power Analysis and how an attack is made
* What are the targets
* What are the tools

## 📑 Table of Contents
1. [What is Side-Channel Analysis? ❓](#sca)
2. [What is leakage? 💧](#leakage)
3. [Example of leaking information 🤏](#example)
4. [SCA in numbers 📊](#numbers)
    1. Breaking `AES-128`
    2. Breaking `AES-256`
5. [Why SCA is unavoidable? ☠️](#unavoidable)
    1. Are we doomed?
6. [What are the targets of SCA? 📌](#targets)
7. [How does Power Analysis work? ⚡](#how_does_it_work)
8. [How to record the power consumption of a device? ⏺️](#trace_capture)
    1. Oscilloscopes and shunt resistors
    2. Some extras!
9. [What tool are we going to use? 🧰](#chipwhisperer)
    1. Documentation!
10. [Conclusions](#conclusions)

## 1) What is Side-Channel Analysis? ❓ <a class="anchor" id="sca"></a>

Side-Channel Analysis (briefly referred to `SCA`) is probably the worst enemy to cryptographic algorithms. In theory and in an ideal world, it should not. 
>Actually, SCA shouldn't even exist in an **ideal** world.

However, in practice, and in the **real** world we all live in, **SCA is one of the most efficient and suitable attacks you may want to use (or fear) against a cryptographic implementation.**

Citing directly from [Wikipedia](https://en.wikipedia.org/wiki/Side-channel_attack):

> *In computer security, a side-channel attack is any attack based on information gained from the implementation of a computer system, rather than weaknesses in the implemented algorithm itself (e.g. cryptanalysis and software bugs). Timing information, power consumption, electromagnetic leaks or even sound can provide an extra source of information, which can be exploited.*

The "trick" to better understand SCA is to properly concentrate on the "Side-Channel" part of the name.

> Since most of us are Italian, let's not-so-correctly translate `Side-Channel Analysis` to `analisi tramite strade secondarie` or `analisi per vie traverse`. Should be clearer now, right?

As stated in the Wiki definition, an attacker (let's call her "Eve") leveraging SCA to gain sensitive information does not use conventional attacks such as cryptanalysis or brute force. On the contrary, she tries to find unconventional ways to leverage  clues one can only exploit in a real world.

These clues are called "leakage", and are all physically emitted by **working** devices.

## 2) What kind of leakage are we talking about? 💧 <a class="anchor" id="leakage"></a>

Take your laptop, for example. Try to leave it idling for 5 minutes. Done? Now touch the palmrest with your hand. It should be cold, or at least a little bit warm. Now try to game on it, for another 5 minutes. Repeat the experiment with your hand. I'm pretty sure now the palmrest is way hotter, maybe even uncomfortable.

> *Congratulations! You just found your first leakage of information from your own very laptop!*

The temperature of the palmrest is in fact telling us, in a very rough way, that the laptop just completed a very demanding operation, something that may require many difficult computations, like the ones required by videogames. If I were to plot the temperature measured I could tell at what point exactly you started and then closed the game.

In general, the most common (but not limited to) information an attacker can exploit is:

1) **Timing information**:

> *How many seconds the completion of this cryptographic implementation is taking? Does the time vary with the secret key used?*
    
2) **Power consumption and electromacnetic leaks**

> *How many Watts a processor is consuming to complete an operation? Can I record a graph with an oscilloscope representing the power consumption profile drawn by the processor over time? And can I retrieve information about the cryptographic key just by looking at this graph?*

3) **Sound emissions**

> *Can I understand what a processor is doing by listening to the [coil whine](https://www.youtube.com/watch?v=T7HsXHqtxrI) emitted?*

Here's a small image to help you summarize this rationale:

<img src="./img/02-Intro to SCA/SCA_intro.png" alt="SCA Intro" width=400>

## 3) Can you give me another example? 🤏 <a class="anchor" id="example"></a>

Imagine you are a WW2 spy, typing a top-secret message on your typewriter. Now imagine your enemy found a way to listen and record you typing in your room. Sure they can listen to your voice communications, but they can also listen to the keys you are typing. Given their position in the chassis, all the keys emit slightly different sounds when hit. We can even say each key emits its unique sound.

Now imagine your opponent has the exact same typewriter. Having the recording of you typing, Eve can now try to reconstruct the message, just by discerning the sound of each key you presssed. It may take time, obviously, but **she will eventually succeed**.

> Remember, *eventually* translates in `prima o poi`, not in `eventualmente`).

**ℹ️ Pro Tip** : If you understand the sound emitted by the space bar (which is quite unique and different from all the others), you can easily find the length of each word typed in the entire message!

## 4) I'm not entirely convinced, can you give me some numbers? 📊 <a class="anchor" id="numbers"></a>

Let's take the [AES-128](https://it.wikipedia.org/wiki/Advanced_Encryption_Standard) block cipher (encryption and decryption algorithm). In an ideal world, Eve can try attacking it by only using [brute-force](https://en.wikipedia.org/wiki/Brute-force_search) or [Differential](https://en.wikipedia.org/wiki/Differential_cryptanalysis) and [Linear Cryptanalysis](https://en.wikipedia.org/wiki/Linear_cryptanalysis). Let's only consider brute-force, which is way easier to understand.

### 4.1) Considering `AES-128`, where the key is expressed on 128 bits.

To brute force it, you will need to try all the $2^{128}$ possible key combinations.

> This is a **HUGE** number:
>
> $2^{128} = 3,402 \cdot 10^{38} = 340\,282\,366\,920\,938\,463\,463\,374\,607\,431\,768\,211\,456$

Now let's assume Eve is in our real, imperfect, world. Contrary to the ideal world, she can now leverage physical emissions, leaked by real cryptographic implementations (i.e. a microprocessor running the `AES-128` algorithm).

With SCA (in particular, Power Analysis), thanks to the leakage information collected, the attack is simplified to testing $2^8 \cdot 2^4 = 2^{12}$ different combinations.

> This is definitely **NOT a huge** number:
> 
> $2^8 \cdot 2^4 = 2^{12} = 4096$

ℹ️ How is that possible? We will see this in the next notebooks, as for now, just trust me.

### 4.2) Considering `AES-256`, where the key is expressed on 256 bits.

When a specific algorithm is considered to be insecure for a very demanding and specific purpose, the usual secure practice is to simply increase the length of the secret key, before evaluating changing the algorithm itself.

What happens if we move from `AES-128` to `AES-256`?

Again, Eve will find the secret key within an extremely reduced number of combinations, that are actually feasible to treat even with a laptop:

* With brute force:
$2^{256} = 1,157 \cdot 10^{77} = \dots $ (you get it, it's huge)

* With SCA:
$2^8 \cdot 2^5 = 2^{13} = 8192$

## 5) Why SCA is unavoidable? ☠️ <a class="anchor" id="unavoidable"></a>
Because we live in a real, imperfect world! Every computing device, in order to move from a computing state to another, has to consume energy. And since [adiabatic state transitions](https://en.wikipedia.org/wiki/Adiabatic_process) and [reversible computing](https://en.wikipedia.org/wiki/Reversible_computing) are not possible, some sort of leakage both in terms of time consumed and thermal radiation will always be present. See the first page of the paper [*Statistics and Secret Leakage* by Coron, Naccache and Kocher](https://dl.acm.org/doi/10.1145/1015047.1015050) if you want to learn more.

This means that, assuming an infinite amount of time and resources, Eve will always be able to collect enough side-channel leakage to guess and disclose your secret key!

### 5.1 Does this mean we are doomed, shall we resign and welcome our leakage overlords?

Not at all! In fact, Eve does not have an infinite amount of time, nor an infinite amount of economical resources.

> ℹ️ As defenders, the goal, when developing countermeasures and protecting our implementation, is to **force the attacker to spend an amount of time and resources as high as possible, so as to make the attack unfeasible or economically unattractive.**

> As an example, do you think Eve will pursue attacking a Bitcoin hardware wallet containing 10.000 dollars worth of BTC if she has to spend 20.000 dollars to hack it?

## 6) What are the targets? 📌 <a class="anchor" id="targets"></a>
With SCA, every **operating** electronic or mechanical device is the target.

> ℹ️ For our purposes, we will only concentrate on electronic circuits, like microcontrollers (think about [*Arduino boards*](https://www.arduino.cc/)), System-on-Chips (like the ones in smart-devices, or automotive applications), general-purpose microprocessors (Intel, AMD).

The sub-branch of SCA attacks we are going to learn is called **Power Analysis** (aka **PA**), i.e. attacks that observe the power consumption of an electronic circuit with the goal of extracting sensitive information from it. The fundamentals of these kind of attacks can also be applied to **ElectroMagnetic Attacks** (aka **EMA**).

> NOTE: It is important to attack **OPERATING** devices: we are going to observe physical clues we can only obtain if the processor is computing something. If the device is `off`, no power hence no computing activity, therefore no clues to extract.
>
> If you don't type on your typewriter, no message is ever going to be exposed!

## 7) How does a Power Analysis attack work? ⚡ <a class="anchor" id="how_does_it_work"></a>
Power Analysis attacks are successfully carried out if the targeted device is in close physical proximity with the attacker. Hence, the devices at risk are all those embeeded microcontrollers, smartcards, IoT devices, hardware access tokens, hardware wallets etc. that permeates our lives. **Basically, every device that can be stolen or momentarily tampered can be a target.**

Imagine Eve stealthy stole your personal smartcard to access the company's main building. Power analysis (or ElectroMagnetic Analysis, which is pretty similar) is probably going to be her goto attack to easily recover the secret key.

Power Analysis works by observing and recording the power consumption profile of the above devices. There are two main class of attacks:

1. [Simple Power Analysis (SPA)](https://en.wikipedia.org/wiki/Power_analysis#Simple_power_analysis)

> In this first attack, the goal of the attacker is (but not limited to) to understand *what is the operation* the microprocessor is performing. The attack is carried out manually: Eve visually inspects the graph of the power consumption and tries to interpret it. Pattern recognition tools and machine learning can be used if the attacker already knows what to look for. Most of the time, a single power consumption trace is sufficient (meaning Eve can succeed even with a single attempt).

> Is there any interesting spikes in the current consumption of the device? Is there any repetitive pattern in the graph? **What useful information can we gain about the code executed by the microprocessor?**

2. [Differential and Correlation Power Analysis (DPA and CPA)](https://en.wikipedia.org/wiki/Power_analysis#Differential_power_analysis)

> This class of attacks is way more complex than SPA, but also way more powerful. DPA and CPA allow to correctly guess the secret keys used by a hardware device. Contrary to SPA, **Eve needs to collect a multitude of power traces**, all referring to the same encryption/decryption algorithm. 

> To do that, Eve needs to trigger multiple times the encryption/decryption operation and record each time the power trace produced by the device. This may take some time, from hours to seconds, depending on the capabilities and on the hardware she disposes. As a rule of thumb: the more traces she captures the more leakage she collects, and the higher are her chances to succeed.

> Once all the necessary traces are captured, she can forget about the device (that may casually re-appear in your pocket "Oh here it is, I thought I lost it!") and proceed with the heavy-lifting analysis. This last step consists of statistical computations, able to refine the key research and finally guess the correct key bytes: more on that later.

## 8) How to record the power consumption profile of a device? ⏺️ <a class="anchor" id="trace_capture"></a>
Let's assume Eve has stolen your [YubiKey](https://en.wikipedia.org/wiki/YubiKey) and she's now back to her main laboratory. How is she going to collect the power traces she need to extract the secret key?

First of all, she needs to crack open the device. Once free from the enclosure, the PCB (printed circuit board) can be examined. The goal is to find the main microcontroller, or the secure element (still a black squared chip) containing the secret keys.

For instance, below you can see the PCB of a Ledger Nano S, a cryptocurrency hardware wallet (courtesy of Karl Kreder and GridPlus, [original resource](https://blog.gridplus.io/hardware-wallet-vulnerabilities-f20688361b88)). The SMT32 is the main microcontroller, responsible for both input and output user interaction, while the ST31 is the secure element in which the keys are stored.

<img src="./img/02-Intro to SCA/ledger_nano_pcb.png" alt="PCB of the Ledger Nano S" width=400>

Once the target microcontroller found, how to proceed recording its current consumption so as to plot the corresponding graph over time?

To do that, Eve needs to have in her lab an oscilloscope.

### 8.1) But oscilloscopes measure the voltage over time, not the current!

Indeed, you are correct. As it is, a plain oscilloscope cannot serve our purpose. But we can adapt it! And a little modification to the device PCB is needed. See the image below.

<img src="./img/02-Intro to SCA/attack_probes.png" alt="PCB modifications" width=600>

Ohm's law comes to our rescue. So as a little shunt resistance $R_s$ 

> $V = R \cdot i$

What Ohm's law tells us is that Voltage and Current are, mathematically, nearly the same thing. The two quantities are tied together by a value, the resistance value. If such value is constant, a variation of the current causes a (scaled) variation of the voltage, equal in sign. This means that if the current consumption spikes, also the voltage spikes.

This is the role played by our shunt resistance. As you can see from the image above, we place the shunt in series to the target microcontroller. 

By placing it like in the image above, the shunt creates a (hopefully small) voltage drop before the microcontroller. We call such voltage drop $V_s$:

$V_s = R_s \cdot i$

We don't want this drop to be too high: we want the resistance value $R_s$ to be quite small, basically insignificant if compared to the equivalent resistance of the microcontroller $R_{\mu P}$. If the $R_s$ value is too high the microprocessor may not have enough voltage and power to run!

Now that we have a properly chosen, constant value for $R_s$, we have basically solved the problem! The current consumption $i$ of the microcontroller is now "translated" into a proper voltage $V_s$, which is now measureable using the probes of our oscilloscope!

Here's an image representing a power trace profile, captured from a running microcontroller, using an oscilloscope and a shunt resistor:

<img src="./img/02-Intro to SCA/power_trace.png" alt="Example of a power trace" width=500>

### 8.2) Some extras!

* [MUST SEE] [YouTube] [A video from LiveOverflow](https://www.youtube.com/watch?v=bFfyROX7V0s) summarizing what we just said!
* [Optional] [Article] [An example of an ElectroMagnetic attack on a Google Titan 2FA key](https://arstechnica.com/information-technology/2021/01/hackers-can-clone-google-titan-2fa-keys-using-a-side-channel-in-nxp-chips/)
    > Did I told you that Electromagnetic and Power Analysis share extremely similar principles?

## 9) What tool are we going to use? 🧰 <a class="anchor" id="chipwhisperer"></a>
Power Analysis attacks became famous at least 20 years ago, (probably) with the paper [*Differential Power Analysis* by Paul Kocher, Joshua Jaffe and Benjamin Jun](https://www.paulkocher.com/doc/DifferentialPowerAnalysis.pdf), which is now a MUST read for every researcher in the field. From then on, the hardware setup used by researchers and hackers has remained more or less the same. Recently, [acquisition cards](https://www.picotech.com/products/oscilloscope) have been preferred to oscilloscopes, due to their increased portability and similar capabilities.

Some years ago (I believe 2014-2015, but don't quote me on that) [Colin O'Flynn](https://colinoflynn.com/about-me/) started his company [NewAE Technology](https://www.newae.com/about). Their main product is the [ChipWhisperer](https://www.newae.com/chipwhisperer) family of boards.

ChipWhisperer is probably the first acquisition board specifically made to target side-channel attacks and fault-injection attacks. If not the first, it is definitely the most famous and supported one.

Quoting directly from NewAE's site:

> *ChipWhisperer(-Lite) is a single board that integrates high-speed power measurement (replacing your oscilloscope), a target device, programmer for the target device, a platform for performing fault injection, and much more. These boards are perfect if you want to teach yourself (or your class) all about side-channel power analysis & fault injection with minimal fuss.*

**REMEMBER: A ChipWhisperer board ships both an integrated capture board and a target board (the microcontroller under attack)! So it's basically a 2-in-1 board: this is incredibly compact and cheap if compared to a full-fledge oscilloscope.**

For instance, this is the board I used for my thesis, the same I'm currently using for my reasearch:

<img src="./img/02-Intro to SCA/CW_lite.png" alt="ChipWhisperer Lite" width=500>

And this one is the board connected to our servers, the one we will use starting from the next notebook!

<img src="./img/02-Intro to SCA/CW_nano.png" alt="ChipWhisperer Nano" width=500>

### 9.1) Documentation! 📔
Since the ChipWhisperer project is incredibly well documented and there's no particular need in reinventing the wheel, I'll leave you below all the documentation you need to understand in details what is ChipWhisperer.

* [MUST READ] [Hardware page of the ChipWhidsperer Nano board](https://rtfm.newae.com/Capture/ChipWhisperer-Nano/)
* [MUST READ] [Software page for all the ChipWhisperer boards](https://chipwhisperer.readthedocs.io/en/latest/getting-started.html)
* [Optional] [Hardware page of the Nano's target board](https://rtfm.newae.com/Targets/CW1101%20Target%20Board/)
* [Optional funny video] [YouTube] [Colin OFlynn introduces the ChipWhisperer-Lite board + DOG!](https://www.youtube.com/watch?v=nU-XGThSn50)

# 10) Conclusions <a class="anchor" id="conclusions"></a>

This marks the end of this tutorial! Now you can head to the next one on Simple Power Analysis

Do you have suggestions, doubts or simply need to reach me?
* 🐦 Twitter: `[at]mrcuve0`
* 💬 Discord: `Mrcuve0#4179`
* 💻 GitHub:  `Mrcuve0`
* 📬 Email: `mrcuve0 [at] posteo [d.ot] net`
* 💼 LinkedIn:
> Please, send me a message via the previous methods before asking for my LinkedIn

---
Copyright ©️ [CINI - Cybersecurity National Lab](https://cybersecnatlab.it/) - Torino, 2022.

> This material is a derivative work of [NewAE's official Jupyter Notebooks](https://github.com/newaetech/chipwhisperer-jupyter), distributed under the open-source GPL license. You can distribute and modify this material (even for commercial trainings), provided you maintain references to this repository and the original authors, and also offer your derived material under the same conditions.


ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

All other product names, logos, and brands are property of their respective owners.

The software is provided 'as is', without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, arising from, out of or in connection with the software or the use or other dealings in the software.